In [2]:
#Capstone project 
#import libraries
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as npl
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [3]:
#import dataset
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

--2019-12-10 12:46:45--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

100%[======================================>] 23,101      --.-K/s   in 0.002s  

2019-12-10 12:46:45 (12.2 MB/s) - ‘loan_train.csv’ saved [23101/23101]



In [4]:
loan = pd.read_csv('loan_train.csv')
loan.head(10)

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male
5,7,7,PAIDOFF,1000,30,9/9/2016,10/8/2016,36,college,male
6,8,8,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,male
7,9,9,PAIDOFF,800,15,9/10/2016,9/24/2016,26,college,male
8,10,10,PAIDOFF,300,7,9/10/2016,9/16/2016,29,college,male
9,11,11,PAIDOFF,1000,15,9/10/2016,10/9/2016,39,High School or Below,male


In [5]:
print(loan.shape)
loan['due_date'] = pd.to_datetime(loan['due_date'])
loan['effective_date'] = pd.to_datetime(loan['effective_date'])
print(loan['Gender'].value_counts())
print(loan['education'].value_counts())

(346, 10)
male      294
female     52
Name: Gender, dtype: int64
High School or Below    151
college                 149
Bechalor                 44
Master or Above           2
Name: education, dtype: int64


In [6]:
#since there are only 2 rows for Master or above, it is not fit for training,so we will drop these 2 rows in training
loan.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

Gender  loan_status
female  PAIDOFF        0.865385
        COLLECTION     0.134615
male    PAIDOFF        0.731293
        COLLECTION     0.268707
Name: loan_status, dtype: float64

In [7]:
# looks like 86% of females payoff loan comparing to 73% of males
#see what day mostly is effective in loan grants
loan['dayofweek'] = loan['effective_date'].dt.dayofweek
#return day of the week
loan['dayofweek'].value_counts()

6    166
0     87
5     31
2     31
1     20
4      8
3      3
Name: dayofweek, dtype: int64

In [8]:
#loans are approved mostly at start and end of week and few in the middle of week
#convert gender to binary
loan['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
loan.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender,dayofweek
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,0,3
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,1,3
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,0,3
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,1,4
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,0,4


In [9]:
loan.groupby(['education'])['loan_status'].value_counts(normalize=True)

education             loan_status
Bechalor              PAIDOFF        0.750000
                      COLLECTION     0.250000
High School or Below  PAIDOFF        0.741722
                      COLLECTION     0.258278
Master or Above       COLLECTION     0.500000
                      PAIDOFF        0.500000
college               PAIDOFF        0.765101
                      COLLECTION     0.234899
Name: loan_status, dtype: float64

In [10]:
#use one hot encoder for education column
Feature = loan[['Principal','terms','age','Gender']]
Feature = pd.concat([Feature,pd.get_dummies(loan['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()

,Principal,terms,age,Gender,Bechalor,High School or Below,college
0,1000,30,45,0,0,1,0
1,1000,30,33,1,1,0,0
2,1000,15,27,0,0,0,1
3,1000,30,28,1,0,0,1
4,1000,30,29,0,0,0,1


In [11]:
#get dependent labels
y = loan['loan_status'].values

In [12]:
#normalize data for better extraction
X = Feature
X_train= preprocessing.StandardScaler().fit(X).transform(X)
X_train[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()


array([[ 0.51578458,  0.92071769,  2.33152555, -0.42056004, -0.38170062,
         1.13639374, -0.86968108],
       [ 0.51578458,  0.92071769,  0.34170148,  2.37778177,  2.61985426,
        -0.87997669, -0.86968108],
       [ 0.51578458, -0.95911111, -0.65321055, -0.42056004, -0.38170062,
        -0.87997669,  1.14984679],
       [ 0.51578458,  0.92071769, -0.48739188,  2.37778177, -0.38170062,
        -0.87997669,  1.14984679],
       [ 0.51578458,  0.92071769, -0.3215732 , -0.42056004, -0.38170062,
        -0.87997669,  1.14984679]])

In [13]:
#Training set is ready, now use various ML algos
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (276, 7) (276,)
Test set: (70, 7) (70,)


In [14]:
# Modeling using knn
#iterate using different values for k and find the best k value
from sklearn.neighbors import KNeighborsClassifier
Ks=15
mean_acc=np.zeros((Ks-1))
std_acc=np.zeros((Ks-1))
ConfustionMx=[];
for n in range(1,Ks):
    #Train Model and Predict  
    KNN = KNeighborsClassifier(n_neighbors=n).fit(X_train,y_train)
    yhat = KNN.predict(X_test)  
    mean_acc[n-1]=np.mean(yhat==y_test);
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
mean_acc

array([0.6       , 0.51428571, 0.68571429, 0.64285714, 0.75714286,
       0.72857143, 0.72857143, 0.7       , 0.72857143, 0.72857143,
       0.72857143, 0.72857143, 0.75714286, 0.72857143])

In [15]:
#from mean accuracy values, it is clear for k=5 accuracy is highest (75.7%)
# Building the model again, using k=5
from sklearn.neighbors import KNeighborsClassifier
k = 5
#Train Model and Predict  
KNN = KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
# evaluate KNN
knn_yhat = KNN.predict(X_test)

In [16]:
#import scoring metrics
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [17]:
# a dictionary for keeping all scores of the classifiers
print(y_test.shape, knn_yhat.shape)
Score_j={}
Score_f={}
Score_j['KNN'] = round(jaccard_similarity_score(y_test, knn_yhat),2)
Score_f['KNN']= round(f1_score(y_test, knn_yhat, average='weighted'), 2)

(70,) (70,)


In [18]:
#DECISION TREE 
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_train, y_train)
y_pred=clf_tree.predict(X_test)

In [19]:
Score_j['Tree'] = round(jaccard_similarity_score(y_test, y_pred),2)
Score_f['Tree'] = round(f1_score(y_test, y_pred, average='weighted'), 2)

In [25]:
from sklearn import svm
SVM = svm.SVC(gamma='auto')
SVM.fit(X_train, y_train)
svm_yhat = SVM.predict(X_test)

In [21]:
Score_j['SVM']=round(jaccard_similarity_score(y_test, svm_yhat),2)
Score_f['SVM'] = round(f1_score(y_test, svm_yhat, average='weighted'), 2)

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.001,solver='lbfgs').fit(X_train,y_train)
lr_yhat = LR.predict(X_test)
lr_yprob= LR.predict_proba(X_test)

In [27]:
Score_j['LogReg'] = round(jaccard_similarity_score(y_test, lr_yhat),2)
Score_f['LogReg'] = round(f1_score(y_test, lr_yhat, average='weighted'),2 )
Score_loss={}
Score_loss['LogReg'] = round(log_loss(y_test, lr_yprob),2)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
df = pd.DataFrame([Score_j,Score_f,Score_loss],index=['Jaccard','f1-Score','LogLoss'])
df

,KNN,LogReg,SVM,Tree
Jaccard,0.76,0.79,0.74,0.64
f1-Score,0.71,0.69,0.67,0.65
LogLoss,NaN,0.54,NaN,NaN
